In [3]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import pyfolio as pf
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [62]:

class St(bt.Strategy):
    params = dict(
        p1=10,
        p2=30,
        limit=0.05,
        limdays=1,
        limdays2=100,
        hold=10,
        usebracket=False,  # use order_target_size
        switchp1p2=False,  # switch prices of order1 and order2
    )
    def notify_order(self, order):
        print('{}   {}: Order ref: {} / Type {} / Status {}'.format(
            self.data.datetime.date(0),self.data.datetime.time(0),
            order.ref, 'Buy' * order.isbuy() or 'Sell',
            order.getstatusname()))

        if order.status == order.Completed:
            self.holdstart = len(self)

        if not order.alive() and order.ref in self.orefs:
            self.orefs.remove(order.ref)

    def __init__(self):
        emafast = bt.ind.EMA(period=self.p.p2)
        emaslow = bt.ind.EMA(period=self.p.p1)
        self.cross = bt.ind.CrossOver(emafast,emaslow)
        

        self.orefs = list()

        if self.p.usebracket:
            print('-' * 5, 'Using buy_bracket')

    def next(self):
        if self.orefs:
            return  # pending orders do nothing
        if (self.data.datetime.time() < datetime.time(9,30)) or (self.data.datetime.time() > datetime.time(15,15)):
            return
        if not self.position:
            if self.cross > 0.0:  # crossing up

                close = self.data.close[0]
                p1 = close * (1.0 - self.p.limit)
                p2 = p1 - 0.02 * close
                p3 = p1 + 0.02 * close

                valid1 = datetime.timedelta(self.p.limdays)
                valid2 = valid3 = datetime.timedelta(self.p.limdays2)

                if self.p.switchp1p2:
                    p1, p2 = p2, p1
                    valid1, valid2 = valid2, valid1

                if not self.p.usebracket:
                    o1 = self.buy(exectype=bt.Order.Limit,
                                  price=p1,
                                  valid=valid1,
                                  transmit=False)

                    print('{}   {} : Oref {} / Buy at {}'.format(
                        self.datetime.date(),self.datetime.time(), o1.ref, p1))

                    o2 = self.sell(exectype=bt.Order.Stop,
                                   price=p2,
                                   valid=valid2,
                                   parent=o1,
                                   transmit=False)

                    print('{}   {}: Oref {} / Sell Stop at {}'.format(
                        self.datetime.date(),self.data.datetime.time(), o2.ref, p2))

                    o3 = self.sell(exectype=bt.Order.Limit,
                                   price=p3,
                                   valid=valid3,
                                   parent=o1,
                                   transmit=True)

                    print('{}   {}: Oref {} / Sell Limit at {}'.format(
                        self.datetime.date(),self.data.datetime.time(), o3.ref, p3))

                    self.orefs = [o1.ref, o2.ref, o3.ref]

                else:
                    os = self.buy_bracket(
                        price=p1, valid=valid1,
                        stopprice=p2, stopargs=dict(valid=valid2),
                        limitprice=p3, limitargs=dict(valid=valid3),)

                    self.orefs = [o.ref for o in os]

        else:  # in the market
            if (len(self) - self.holdstart) >= self.p.hold:
                pass  # do nothing in this case


In [52]:

#A function for properly processing and printing analyzers
def printTradeAnalysis(analyzer):
    
    #Get the results we are interested in
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    
    #Designate the rows
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    #Check which set of headers is the longest.
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    #Print the rows
    
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("\n\nTrade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

In [65]:
if __name__ == '__main__':
    orig_stdout = sys.stdout
    f = open('/home/lokendra/Downloads/intraday_output.txt', 'w')
    sys.stdout = f
    cerebro = bt.Cerebro()
    cerebro.addstrategy(St)
 
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    #modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    #datapath = os.path.join(modpath, 'datas/orcl-1995-2014.txt')
    datapath = '/home/lokendra/Downloads/Nifty_2013.csv'

    #Check the format and positions of the datafeeds in case another file is used
    data = bt.feeds.GenericCSVData(
        dataname=datapath,
        
        #Even though the data file has 2013 in name, the data is actually of 2015
        fromdate=datetime.datetime(2015, 1, 1),
        todate=datetime.datetime(2015, 1, 31),
        nullvalue = 0.0,
        
        dtformat = ('%d/%m/%Y'),
        tmformat = ('%H:%M:%S'),
        
        timeframe = bt.TimeFrame.Minutes,
        compression = 1,
        #Compression is useful if the data is per minute but we want to use 5 min intervals
                
        datetime = 0,
        time = 1,
        open = 2,
        high = 3,
        low = 4,
        close = 5,
        volume = 6,
        openinterest = -1,
        reverse=False)

    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)

    cerebro.addsizer(bt.sizers.FixedSize, stake=10)
    
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

    cerebro.broker.setcommission(commission=0.001)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    strats = cerebro.run()
    printTradeAnalysis(strats[0].analyzers.ta.get_analysis())

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    sys.stdout = orig_stdout
    f.close()
    
    #A weird error, scroll down and check. After september 8th all orders are displaying margin/cancelled

In [34]:

cerebro.plot()